#Load Data

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.4.1


In [ ]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import cv2
from keras.models import model_from_json
from keras.preprocessing import image

In [ ]:
pwd

'/content/gdrive/My Drive/Kaggle/Percobaan 6'

In [ ]:
# load model
model = load_model('model.h5')

In [ ]:
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 47, 47, 32)        160       
_________________________________________________________________
batch_normalization (BatchNo (None, 47, 47, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        8256      
_________________________________________________________________
batch_normalization_1 (Batch (None, 22, 22, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 64)        0

In [ ]:
# load dataset
#Importing Data from CSV file
fer2013=pd.read_csv("fer2013.csv")
labels=fer2013.iloc[:,[0]].values
pixels=fer2013['pixels']

In [ ]:
#Facial Expressions
Ekspresi={0:"Marah",1:"Jijik",2:"Takut",3:"Senang",4:"Sedih",5:"Terkejut",6:"Netral"}
from keras.utils import to_categorical 
labels = to_categorical(labels,len(Ekspresi))

In [ ]:
#converting pixels to Gray Scale images of 48X48 
image = np.array([np.fromstring(pixel, dtype=int, sep=" ")for pixel in pixels])
image = image/255.0
image = image.reshape(image.shape[0],48,48,1).astype('float32')

In [ ]:
#splitting data into training and test data
train_features,test_features,train_labels,test_labels = train_test_split(image,labels,test_size=0.2,random_state=0)

#Test Vidio

In [ ]:
import cv2
def make_prediction(pred):
    pred = cv2.resize(pred,(48,48))
    pred = pred/255.0
    pred = np.array(pred).reshape(-1,48,48,1)
    predict = np.argmax(model.predict(pred),axis = 1)
    return predict[0]

In [ ]:
def Emotion_in_video():
    face_Emotion = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    capture=cv2.VideoCapture(0)
    while True:
        ret,img=capture.read()  
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        face = face_Emotion.detectMultiScale(gray,1.3,5)
        
        for (x,y,w,h) in face:
            sub_face = gray[y:y+h, x:x+w]
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            res = make_prediction(sub_face)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img,str(Ekspresi[res]),(x,y-5),font,0.5,(205,200,50),1,cv2.LINE_AA)
        cv2.imshow('img',img)
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
Emotion_in_video()